In [ ]:
import numpy as np
from scipy import stats

# ------------------------------------------------------------
# آزمون کای‌دو نیکویی برازش (Chi-square goodness-of-fit)
# ------------------------------------------------------------
class ChiSquareGoodnessOfFit:
    def __init__(self, observed, expected=None, probs=None, ddof=0):
        """
        observed: آرایه‌ای از فراوانی‌های مشاهده‌شده
        expected: (اختیاری) فراوانی‌های مورد انتظار (در صورت داده نشدن از probs محاسبه می‌شود)
        probs: (اختیاری) احتمالات نظری برای هر دسته (اگر expected داده نشود)
        ddof: تعدیل درجه آزادی (معمولاً 0)
        """
        self.observed = np.asarray(observed)
        self.n = np.sum(self.observed)
        self.ddof = ddof

        if expected is not None:
            self.expected = np.asarray(expected)
            if len(self.observed) != len(self.expected):
                raise ValueError("طول observed و expected باید برابر باشد")
            self.probs = self.expected / np.sum(self.expected)
        elif probs is not None:
            self.probs = np.asarray(probs)
            if len(self.observed) != len(self.probs):
                raise ValueError("طول observed و probs باید برابر باشد")
            if not np.isclose(np.sum(self.probs), 1):
                raise ValueError("مجموع probs باید 1 باشد")
            self.expected = self.probs * self.n
        else:
            # فرض توزیع یکنواخت
            self.probs = np.ones_like(self.observed) / len(self.observed)
            self.expected = self.probs * self.n

        self._compute()

    def _compute(self):
        # جلوگیری از انتظار صفر
        if np.any(self.expected == 0):
            raise ValueError("مقدار مورد انتظار نمی‌تواند صفر باشد")
        self.chi2_stat = np.sum((self.observed - self.expected) ** 2 / self.expected)
        self.df = len(self.observed) - 1 - self.ddof
        self.p_value = 1 - stats.chi2.cdf(self.chi2_stat, self.df)

    def summary(self, alpha=0.05):
        print("آزمون کای‌دو نیکویی برازش (Chi-square goodness-of-fit)")
        print("دسته‌ها: ", list(range(len(self.observed))))
        print("مشاهده‌شده: ", self.observed)
        print("موردانتظار: ", self.expected)
        print(f"آماره کای‌دو = {self.chi2_stat:.4f}")
        print(f"درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (توزیع مطابقت ندارد) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توزیع مطابقت دارد) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کولموگروف-اسمیرنوف یک نمونه‌ای (Kolmogorov–Smirnov one-sample test)
# ------------------------------------------------------------
class KolmogorovSmirnovOneSample:
    def __init__(self, data, cdf='norm', args=(), alternative='two-sided'):
        """
        data: آرایه داده‌ها
        cdf: نام توزیع (رشته) یا تابع CDF
            پشتیبانی از: 'norm', 'expon', 'uniform', 'lognorm', و ...
        args: پارامترهای توزیع (به جز پارامتر مکان و مقیاس که در صورت لزوم مشخص شود)
        alternative: 'two-sided', 'less', 'greater'
        """
        self.data = np.asarray(data)
        self.alternative = alternative

        if isinstance(cdf, str):
            # توزیع‌های رایج
            dist_map = {
                'norm': stats.norm,
                'expon': stats.expon,
                'uniform': stats.uniform,
                'lognorm': stats.lognorm,
                'gamma': stats.gamma,
                'beta': stats.beta,
            }
            if cdf not in dist_map:
                raise ValueError(f"توزیع {cdf} پشتیبانی نمی‌شود. از توابع stats استفاده کنید.")
            self.dist = dist_map[cdf]
            self.cdf_func = lambda x: self.dist.cdf(x, *args)
        elif callable(cdf):
            self.cdf_func = cdf
        else:
            raise ValueError("cdf باید رشته (نام توزیع) یا تابع CDF باشد")

        self._compute()

    def _compute(self):
        self.statistic, self.p_value = stats.ks_1samp(self.data, self.cdf_func, alternative=self.alternative)

    def summary(self, alpha=0.05):
        print("آزمون کولموگروف-اسمیرنوف یک نمونه‌ای (Kolmogorov-Smirnov)")
        print(f"تعداد داده: {len(self.data)}")
        print(f"آماره KS = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (توزیع مطابقت ندارد) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توزیع مطابقت دارد) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون اندرسون-دارلینگ (Anderson–Darling test)
# ------------------------------------------------------------
class AndersonDarlingTest:
    def __init__(self, data, dist='norm'):
        """
        data: آرایه داده‌ها
        dist: نام توزیع ('norm', 'expon', 'logistic', 'gumbel')
        """
        self.data = np.asarray(data)
        self.dist = dist
        self._compute()

    def _compute(self):
        self.result = stats.anderson(self.data, dist=self.dist)
        self.statistic = self.result.statistic
        # مقادیر بحرانی و سطح معنی‌داری
        self.critical_values = self.result.critical_values
        self.significance_level = self.result.significance_level

    def summary(self, alpha=0.05):
        print("آزمون اندرسون-دارلینگ (Anderson-Darling)")
        print(f"توزیع مورد آزمون: {self.dist}")
        print(f"تعداد داده: {len(self.data)}")
        print(f"آماره AD = {self.statistic:.4f}")
        print("مقادیر بحرانی و سطوح معنی‌داری:")
        for sl, cv in zip(self.significance_level, self.critical_values):
            print(f"  سطح {sl}%: {cv:.4f}")
        # تعیین نتیجه در سطح آلفا
        # توجه: significance_level بر حسب درصد است (15,10,5,2.5,1). آلفا باید به درصد تبدیل شود
        alpha_percent = alpha * 100
        # پیدا کردن نزدیکترین سطح
        idx = np.argmin(np.abs(self.significance_level - alpha_percent))
        critical = self.critical_values[idx]
        if self.statistic > critical:
            print(f"نتیجه: رد فرض صفر در سطح {alpha} (آماره بزرگتر از مقدار بحرانی {critical:.4f})")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha} (آماره کوچکتر از مقدار بحرانی {critical:.4f})")


# ------------------------------------------------------------
# آزمون شاپیرو-ویلک (Shapiro–Wilk test for normality)
# ------------------------------------------------------------
class ShapiroWilkTest:
    def __init__(self, data):
        """
        data: آرایه داده‌ها
        """
        self.data = np.asarray(data)
        self._compute()

    def _compute(self):
        self.statistic, self.p_value = stats.shapiro(self.data)

    def summary(self, alpha=0.05):
        print("آزمون شاپیرو-ویلک (Shapiro-Wilk) برای نرمال بودن")
        print(f"تعداد داده: {len(self.data)}")
        print(f"آماره W = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (داده‌ها نرمال نیستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (داده‌ها نرمال هستند) در سطح {alpha}")